In [70]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense,Dropout

In [71]:
data = pd.read_csv("job_level.csv")
data.head()

,age,education_level,years_experience,num_projects,certifications,github_contributions,hacker_rank_score,job_level
0,58,1,15,19,2,287,51,1
1,48,0,13,28,1,976,46,2
2,34,1,15,3,8,104,6,1
3,27,2,4,6,0,227,10,2
4,40,2,8,27,6,246,60,1


In [72]:
data.describe()

,age,education_level,years_experience,num_projects,certifications,github_contributions,hacker_rank_score,job_level
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,39.991000,0.98700,9.389000,25.126000,4.483000,517.155000,49.902000,0.977000
std,11.780055,0.81578,5.795107,14.369224,2.876454,292.817218,28.702931,0.800695
min,20.000000,0.00000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
25%,30.000000,0.00000,4.000000,12.000000,2.000000,251.750000,25.000000,0.000000
50%,41.000000,1.00000,9.000000,26.000000,5.000000,521.500000,50.000000,1.000000
75%,50.250000,2.00000,14.000000,37.000000,7.000000,784.000000,76.000000,2.000000
max,59.000000,2.00000,19.000000,49.000000,9.000000,999.000000,99.000000,2.000000


In [73]:
X = data.drop(columns=['job_level'], axis=1)
X

,age,education_level,years_experience,num_projects,certifications,github_contributions,hacker_rank_score
0,58,1,15,19,2,287,51
1,48,0,13,28,1,976,46
2,34,1,15,3,8,104,6
3,27,2,4,6,0,227,10
4,40,2,8,27,6,246,60
...,...,...,...,...,...,...,...
995,27,1,18,28,0,576,0
996,47,0,19,23,3,866,60
997,50,0,7,23,9,231,13
998,28,2,18,41,3,327,51


In [74]:
y = data['job_level'].astype('int') 
y.head()

0    1
1    2
2    1
3    2
4    1
Name: job_level, dtype: int64

In [75]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [76]:
x_train.shape,y_test.shape

((800, 7), (200,))

In [77]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [78]:
x_train_scaled.shape, x_test_scaled.shape

((800, 7), (200, 7))

In [79]:
num_classes = y.nunique()

In [ ]:
model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(x_train_scaled.shape[1],)), 
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
     Dense(32, activation='relu'),
    Dropout(0.4),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,691 (10.51 KB)

 Trainable params: 2,691 (10.51 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
log_dir = "tensorflowlogs"
tensorboard_cb = TensorBoard(log_dir = log_dir)

In [83]:
early_stop_cb =  EarlyStopping(monitor='val_loss', patience=5,verbose=1)

In [84]:
checkpoint_cb = ModelCheckpoint(filepath='best_model.h5',monitor='val_loss',save_best_only=True,save_freq=10)

In [86]:
hist = model.fit(
    x_train_scaled, y_train,
    epochs=50,
    callbacks=[tensorboard_cb,checkpoint_cb],
    validation_split=0.1
)
#  early_stop_cb, 

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.3786 - loss: 1.0918 - val_accuracy: 0.3750 - val_loss: 1.0873
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.3836 - loss: 1.0910 - val_accuracy: 0.3625 - val_loss: 1.0944
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4174 - loss: 1.0783 - val_accuracy: 0.3875 - val_loss: 1.0974
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3776 - loss: 1.0942 - val_accuracy: 0.3875 - val_loss: 1.1025
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4243 - loss: 1.0775 - val_accuracy: 0.3500 - val_loss: 1.1043
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.4107 - loss: 1.0866 - val_accuracy: 0.3500 - val_loss: 1.0979
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.4067 - loss: 1.0741 - val_accuracy: 0.3500 - val_loss: 1.1041
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3668 - loss: 1.0939 - val_accuracy: 0.2875 - v